# Poisson Blending
Contexto breve: Recordemos la primera vez que intentamos mezclar 2 imagenes que no tienen relación alguna, es probable que el resultado no fuera muy agradable, esto se debe a que las intensidades entre cada imagen puede que sean muy distintas entre si, creando un cambio bruzco entre la imagen de fondo y la que esta encima de.

Existe un método para "mezclar" una imagen fuente dentro de una imagen objetivo, conocido como Poisson Blending, aunque también es conocido como Gradient Domain Image Blending (Mezcla de imágenes en el dominio del gradiente) debido al uso de este operador.

# Explicación simplificada para Poisson Blending
El método es relativamente sencillo, sin embargo, las justificaciones mátematicas pueden llegar asustar un poco si se tiene poca experiencia en cálculo multidimensional. Por lo tanto, harémos una explicación un poco más simplificada para explicar el comportamiento general del algoritmo, enfocandonos simplemente en un vector de intensidades (una fila de pixeles, en lugar de una imagen de 2 dimensiones).

## Modelado y definiciones importantes
Antes de entrar de lleno al algoritmo, es importante hablar un poco de las bases de este modelo. Algunas palabras que usaremos son:
- Imagen objetivo: Representa la imagen que se encontrará en el fondo
- Imagen fuente: Representa la imagen que se colocará sobre la imagen fuente
- Intensidad: Cantidad de que tanto brilla un pixel (si solo esta en escala de grises)
### ¿Qué es el gradiente de una imagen?
El calculo del gradiente es muy importante, por lo que veremos como es que este se puede calcular en el caso de 1 dimensión.

Supongamos que tenemos nuestra imagen en escala de grises:

<img src="./img1.png" alt="imagen_objetivo" width="50%" height="auto"/>

Cada pixel tiene su propio valor de intensidad, de la imagen notamos que tenemos 8 pixeles, los cuales contienen los valores de intensidad $I_1,...,I_8$. Para una mejor visualización de esto, podemos crear la siguiente gráfica discretizada:

<img src="./hist1.png" alt="hist1" width="50%" height="auto"/>

Además, cada pixel tiene su propio valor de gradiente.

Nota:Cuando hablamos de gradiente en 1 dimensión, sabemos que esto se reduce al calculo de la derivada. 

Supongamos que queremos calcular el gradiente del pixel 3, entonces tenemos: 

$$
\frac{I_{3+\Delta p}-I_3}{\Delta p}
$$

Nuestro $\Delta p$ más pequeño es $1$ debido a la distancia entre los pixeles, esto nos deja:

$$
\frac{I_{3+ 1}-I_3}{1}= I_4-I_3=5-3=2
$$

Definimos el vector gradiente entre los pixeles <p,q> como:
$$
v_{p,q}=I_p-I_q = -v_{q,p}
$$


Vemos que el gradiente v_{p,q}, no es más que el cambio entre las intensidades desde q hasta p. Visualmente podemos ver esto como:

<img src="./gradiente.png" alt="gradiente" width="50%" height="auto"/>

### Un pequeño problema
Ahora que sabemos como describir un pixel en terminos de intensidades y gradientes, podemos ver una aplicación sencilla.

Supongamos que por alguna razón, los pixeles 3, 4, 5 y 6 desaparecieron y no sabemos que intensidades tenian. Sin embargo, tenemos guardados los valores de los gradientes que le corresponden a estos 4 pixeles. Además, las intensidades de los pixeles 2 y 7 son conocidas, sabiendo esto, ¿Cuál es el valor de intensidad de los pixeles desaparecidos?

<img src="./hist3.png" alt="gradiente" width="50%" height="auto"/>

Si recordamos la definición del gradiente:
$$
\begin{align}
v_{p,q}&=I_p-I_q\\
I_p &= v_{p,q} + I_q
\end{align}
$$

Sustituyendo tenemos:
$$
\begin{align}
I_3 &= v_{3,2} + I_2\\
I_3 &= -1 + 4\\
I_3 &= 3
\end{align}
$$

Ahora, que conocemos $I_3$, es fácil notar que siguiendo el mismo procedimiento, podemos encontrar $I_4$, y los demás pixeles, consiguiendo reconstruir la imagen original. 
Esto fue gracias a 2 cosas, conociamos los gradientes de los pixeles desconocidos y conociamos los valores de frontera o limites que estaban a la izquierda y derecha de nuestros pixeles perdidos.

Visto de otra forma, para nosotros poder reconstruir una imagen, ocupamos encontrar los valores de intensidad $I_p$, $I_q$, cuya diferencia entre ellos sea minima, respecto a la diferencia de ellos a su gradiente, $(I_p - I_q - v_{p,q})^2$, usamos el cuadrado ya que lo unico que nos importa es que la cantidad sea pequeña. Si la diferencia entre las intensidades, es muy parecida al gradiente entre los pixeles, entonces la imagen será muy similar a la imagen original

En general, una imagen puede ser reconstruida si minimizamos el valor de la función de energía $h$:

$$
h(I_u,...,I_k)= (I_u - I_{(u-1)} - v_{u,u-1})^2 + (I_{(u-1)} - I_u - v_{u-1,u})^2+...+(I_k - I_{(k-1)} - v_{k,k-1})^2 + (I_{(k-1)} - I_k - v_{k-1,k})^2
$$

$$ 
h = \sum_{j=u}^{k} (I_j - I_{(j-1)} - v_{j,j-1})^2 + (I_{(j-1)} - I_j - v_{j-1,j})^2,\quad u,...,k = \text{pixeles a reconstruir}
$$


La función de energia $h$ nos mide que tanto se parecen los gradientes originales con los gradientes nuevos, minimizando esta función conseguimos reconstruir nuestra imagen.

Para nuestro caso particular, la función de energía sería:

$$
\begin{align}
h(I_3,I_4,I_5,I_6)&= (I_3 - I_{2} - v_{3,2})^2 + (I_{2} - I_3 - v_{2,3})^2\\
&+(I_4 - I_{3} - v_{4,3})^2 + (I_{3} - I_4 - v_{3,4})^2\\
&+(I_5 - I_{4} - v_{5,4})^2 + (I_{4} - I_5 - v_{4,5})^2\\
&+(I_6 - I_{5} - v_{6,5})^2 + (I_{5} - I_6 - v_{5,6})^2\\
&+(I_7 - I_{6} - v_{7,6})^2 + (I_{6} - I_7 - v_{6,7})^2
\end{align}
$$

M

Ocupamos minimizar esta función, como simplemente es una función cuadratica, igualamos su derivada a cero respecto a cada pixel a reconstruir, viendo un caso particular, usamos un caso dónde no nos encontramos en la frontera, por ejemplo $I_5$. También como sabemos $v_{p,q} = -v_{q,p}$, tenemos:
$$
\begin{align}
\frac{\partial{h}}{\partial{I_5}} &= 0\\
2(I_5−I_4−v_{5,4})−2(I_4−I_5−v_{4,5})−2(I_6−I_5−v_{6,5})+2(I_5−I_6−v_{5,6}) &= 0\\
I_5−I_4−v_{5,4}−I_4+I_5+v_{4,5}−I_6+I_5+v_{6,5}+I_5−I_6−v_{5,6} &= 0\\
4I_5−2I_6−2I_4−v_{5,4}+v_{4,5}+v_{6,5}−v_{5,6} &= 0\\
4I_5−2I_6−2I_4−2v_{5,4}−2v_{5,6} &= 0\\
2I_5−I_6−I_4−v_{5,4}−v_{5,6} &= 0\\
2I_5−I_6−I_4 &= v_{5,4}+v_{5,6} =-3\\
2I_5−I_6−I_4 &=-3
\end{align}
$$

Tenemos que nuestros valores conocidos, a la derecha de la igualdad, son simplemente nuestros gradientes, mientras que a la izquierda solo tenemos valores desconocidos.

Para el caso donde el pixel a reconstruir es frontera, por ejemplo $I_3$ tenemos:
$$
\begin{align}
\frac{\partial{h}}{\partial{I_3}} &= 0\\
2(I_3−I_2−v_{3,2})−2(I_2−I_3−v_{2,3})−2(I_4−I_3−v_{4,3})+2(I_3−I_4−v_{3,4}) &= 0\\
4I_3−2I_4−2I_2−2v_{3,2}−2v_{3,4} &= 0\\
2I_3−I_4−I_2−v_{3,2}−v_{3,4} &= 0\\
2I_3−I_4−I_2 &= v_{3,2}+v_{3,4}\\
2I_3−I_4 &= v_{3,2}+v_{3,4} + I_2 = 1\\
2I_3−I_4 &=1
\end{align}
$$

Notamos que llegamos a una expresión similar, sin embargo, al ser un pixel de frontero, $I_2$, pertenece a nuestras condiciones de frontera, por lo que la ecuación se simplifica.

Aplicando las demas derivadas, llegamos al siguiente sistema de ecuaciones lineales:

$$
\begin{align}
2I_3−I_4 &=1\\
2I_4−I_3−I_5 &=3\\
2I_5−I_6−I_4 &=-3\\
2I_6−I_5 &=8
\end{align}
$$

Resolviendo este sistema por cualquier método deseado, obtenemos nuestros valores de intensidad perdidos.
# __________________________________________________________

## Aplicación de lo visto para la mezcla de imagenes
Hasta el momento, solo hemos hablado sobre como calcular el gradiente de los pixeles, y un pequeño problema donde se reconstruyeron las intensidades de una imagen, en la cual solo se conocian sus gradientes originales.  
Sin embargo, estos 2 aspectos vistos, son suficientes para mezclar 2 imagenes.

Supongamos que tenemos una imagen fuente (esta será la imagen que queremos recortar y pegar en algun destino)

<img src="./img2.png" alt="img2" width="50%" height="auto"/>

la cual tiene ciertos valores de intensidad, como vemos en la siguiente imagen:

<img src="./Imagen_fuente_1.png" alt="src_img_1" width="50%" height="auto"/>

Definimos como mascara o región fuente, a la sección de imagen que queremos colocar en nuestra imagen destino, en nuestro ejemplo seran los pixeles 3, 4, 5 y 6:

<img src="./Imagen_fuente_2.png" alt="src_img_2" width="50%" height="auto"/>

La imagen destino, es aquella en donde queremos colocar la región fuente que recortamos de nuestra imagen fuente, para nuestro ejemplo usaremos la siguiente imagen:

<img src="./img1.png" alt="img1" width="50%" height="auto"/>

<img src="./img_des_1.png" alt="img_des_1" width="50%" height="auto"/>

Si pegamos las intensidades de los pixeles tal y como estan en la imagen fuente, tendríamos:

<img src="./mezcla1.png" alt="mezcla1" width="50%" height="auto"/>

Es fácil notar que esta imagen, tiene una región con mucho brillo mientras que otra no.  
Viendola desde un gráfico de intensidades, vemos esto aún más claro:

<img src="./mezcla2.png" alt="img1" width="50%" height="auto"/>

Las regiones de alto brillo, pueden llegar a producir un borde "natural" entre la region fuente y la región imagen destino, lo que provoca una visualización poco agradable, parecido a lo que ocurre en la siguiente image

<img src="./x1.jpg" alt="x1" width="50%" height="auto"/>

Esto representa un problema si realmente queremos que nuestra mezcla se vea natural.
## Entonces, ¿Cómo solucionamos estos cambios bruscos de intensidades?
Bueno, si recordamos un poco nuestro problema que análizamos anteriormente, donde teniamos una cierta imagen, la cuál por alguna razón perdía las intensidades en una región, sin embargo conociamos los gradientes de esos pixeles perdidos, es algo que podemos aplica aquí.  
Vamos a suponer que los pixeles de la imagen destino, que corresponden a los pixeles de la región fuente se perdieron, además calcularemos los gradientes de los pixeles de la imagen fuente en la región fuente.  
Siguiendo el mismo procedimiento que en el problema original, formamos la función de energía, donde las intensidades que se encuentran en la frontera perteneciente a la imagen destino, son conocidos, mientras que las intensidades $I_n$ perteneciente a la región fuente, son desconocidas. Además los gradientes corresponden a la imagen fuente, usando las consideraciones anteriores tenemos:

$$
\begin{align}
h(I_3,I_4,I_5,I_6)&= (I_3 - I_{2} - v_{3,2})^2 + (I_{2} - I_3 - v_{2,3})^2\\
&+(I_4 - I_{3} - v_{4,3})^2 + (I_{3} - I_4 - v_{3,4})^2\\
&+(I_5 - I_{4} - v_{5,4})^2 + (I_{4} - I_5 - v_{4,5})^2\\
&+(I_6 - I_{5} - v_{6,5})^2 + (I_{5} - I_6 - v_{5,6})^2\\
&+(I_7 - I_{6} - v_{7,6})^2 + (I_{6} - I_7 - v_{6,7})^2
\end{align}
$$


Minimizando h, encontramos los valores minimos que deben tener los valores de intensidad $I_3, I_4,I_5,I_6$.  
Derivando la función de energía respecto a cada pixel llegamos al sistema de ecuaciones:

$$
\begin{align}
2I_3−I_4 &=10\\
2I_4−I_3−I_5 &=-8\\
2I_5−I_6−I_4 &=9\\
2I_6−I_5 &=1
\end{align}
$$

Cuyas soluciones son:

$$
\begin{align}
I_3 &=7\\
I_4 &=4\\
I_5 &=9\\
I_6 &=5
\end{align}
$$

Cuya visualizaciones son las siguientes:

<img src="./mezcla3.png" alt="mezcla3" width="50%" height="auto"/>

<img src="./mezcla4.png" alt="mezcla4" width="50%" height="auto"/>

In [ ]:
# librerias
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from scipy.sparse import lil_matrix
from scipy.sparse.linalg import spsolve
from matplotlib.colors import ListedColormap

In [ ]:
def poisson_blending(src, mask, target, x0=0, y0=0):
    # --- Me aseguro de los tipos de datos ---
    mask = mask.astype(np.uint8)
    src = src.astype(np.float64)
    target = target.astype(np.float64)

    h_mask, w_mask = mask.shape
    h_target, w_target = target.shape

    # --- Índices de píxeles válidos dentro de la mascara---
    r_idx, c_idx = np.where(mask > 0)
    N = len(r_idx)
    pixel_index = -np.ones_like(mask, dtype=int)
    pixel_index[r_idx, c_idx] = np.arange(N)

    # --- Inicializando variables del sistema AI = B ---
    A = lil_matrix((N, N), dtype=np.float64)
    B = np.zeros(N, dtype=np.float64)

    # --- Construyendo la matriz A y matriz B
    
    for p in range(N):
        i, j = r_idx[p], c_idx[p]
        A[p,p] = 4.0
        sum_v = 0.0
        
        # Vecino arriba y abajo, izquierdo y derecho.
        q_neighbors = [(i-1,j),(i+1,j),(i,j-1),(i,j+1)]

        for nr,nc in q_neighbors:
            if 0 <= nr < src.shape[0] and 0 <= nc < src.shape[1]:
                if pixel_index[nr,nc] != -1:
                    # Es una intensidad desconocida, estamos en el caso:
                    # 4I_n (( - I_v1 )) = ...
                    A[p, pixel_index[nr, nc]] = -1.0
                else:
                    # La intensidad en este caso, al ser frontera, es conocida, por lo tanto se suma 
                    # en el lado derecho:
                    # 4I_n - ... = v_1 + v_2 + v_3 + v_4 + ((  I_vn ))
                    abs_y, abs_x = y0 + nr, x0+ nc
                    if 0 <= abs_y < target.shape[0] and 0 <= abs_x < target.shape[1]:
                        B[p] += target[abs_y,abs_x]
                # Sumamos el gradiente del vecino, estamos haciendo:
                # # 4I_n - ... = (( v_1 + v_2 + v_3 + v_4 )) +  ... 
                sum_v += src[i, j] - src[nr, nc]
            else:
                A[p,p] -= 1
        B[p] += sum_v


    # --- Resolviendo el sistema ---
    blended = target.copy()

    I = spsolve(A.tocsr(), B)
    I = np.clip(I, 0, 255)

    for p in range(N):
        i, j = r_idx[p], c_idx[p]
        blended[y0 + i, x0 + j] = I[p]

    return blended.astype(np.uint8)

In [ ]:
def poisson_blending2(src, mask, target, x0=0, y0=0):
    # --- Me aseguro de los tipos de datos ---
    mask = mask.astype(np.uint8)
    src = src.astype(np.float64)
    target = target.astype(np.float64)

    h_mask, w_mask = mask.shape
    h_target, w_target = target.shape

    # --- Índices de píxeles válidos dentro de la mascara---
    r_idx, c_idx = np.where(mask > 0)
    N = len(r_idx)
    pixel_index = -np.ones_like(mask, dtype=int)
    pixel_index[r_idx, c_idx] = np.arange(N)

    # --- Construir sistema Ax = B ---
    A = lil_matrix((N, N), dtype=np.float64)
    B = np.zeros(N, dtype=np.float64)

    for p in range(N):
        i, j = r_idx[p], c_idx[p]
        A[p,p] = 4.0
        sum_v = 0.0
        
        # Vecino arriba y abajo, izquierdo y derecho.
        q_neighbors = [(i-1,j),(i+1,j),(i,j-1),(i,j+1)]

        for nr,nc in q_neighbors:
            if 0 <= nr < src.shape[0] and 0 <= nc < src.shape[1]:
                if pixel_index[nr,nc] != -1:
                    # Es una intensidad desconocida, estamos en el caso:
                    # 4I_n (( - I_v1 )) = ...
                    A[p, pixel_index[nr, nc]] = -1.0
                else:
                    # La intensidad en este caso, al ser frontera, es conocida, por lo tanto se suma 
                    # en el lado derecho:
                    # 4I_n - ... = v_1 + v_2 + v_3 + v_4 + ((  I_vn ))
                    abs_y, abs_x = y0 + nr, x0+ nc
                    if 0 <= abs_y < target.shape[0] and 0 <= abs_x < target.shape[1]:
                        B[p] += target[abs_y,abs_x]
                # Sumamos el gradiente del vecino, estamos haciendo:
                # # 4I_n - ... = (( v_1 + v_2 + v_3 + v_4 )) +  ... 
                sum_v += src[i, j] - src[nr, nc]
        B[p] += sum_v


        # --- Resolver sistema ---
    blended = target.copy()

    I = spsolve(A.tocsr(), B)
    I = np.clip(I, 0, 255)

    for p in range(N):
        i, j = r_idx[p], c_idx[p]
        blended[y0 + i, x0 + j] = I[p]

    return blended.astype(np.uint8)

In [ ]:
def blend(src, mask, target, x0=0, y0=0):
    # --- Me aseguro de los tipos de datos ---
    mask = mask.astype(np.uint8)
    src = src.astype(np.float64)
    target = target.astype(np.float64)

    h_mask, w_mask = mask.shape
    h_target, w_target = target.shape

    blended = target.copy()

    for i in range(h_mask):
        for j in range(w_mask):
            if mask[i,j] == 1:
                blended[y0+i,x0+j] = src[i,j]

    return blended.astype(np.uint8)

In [ ]:
A = np.array([[0,0,0,0],[0,1,1,0],[0,1,1,0],[0,0,0,0]],dtype=np.uint8)
r_idx, c_idx = np.where(A > 0)
N = len(r_idx)
pixel_index = -np.ones_like(A, dtype=int)
pixel_index[r_idx, c_idx] = np.arange(N)
# B = (
print(A)
print(r_idx)
print(c_idx)
print(N)
print(pixel_index)

for p in range(N):
    i, j = r_idx[p], c_idx[p]
    A[p,p] = 4.0
    sum_v = 0.0
    
    # Vecino arriba y abajo, izquierdo y derecho.
    q_neighbors = [(i-1,j),(i+1,j),(i,j-1),(i,j+1)]
    print(q_neighbors)


In [ ]:
# Ejemplo 1 -----> falta normalizar la mascara, se usa la forma incorrecta
src = cv.imread("./ex1/src1.jpg",cv.IMREAD_GRAYSCALE)
mask = cv.imread("./ex1/mask.jpg",cv.IMREAD_GRAYSCALE)
target = cv.imread("./ex1/target.jpg",cv.IMREAD_GRAYSCALE)

mask = np.clip(mask, 0, 1)

print(src.shape)
print(mask.shape)
print(target.shape)
print(mask.max())

cmap = ListedColormap(['black', 'red'])

plt.subplot(1,3,1)
plt.imshow(src,cmap="gray")
plt.title('Source Image')
plt.axis('off')

plt.subplot(1,3,2)
plt.imshow(mask,cmap=cmap)
plt.title('Mask')
plt.axis('off')

plt.subplot(1,3,3)
plt.imshow(target,cmap="gray")
plt.title('Target Image')
plt.axis('off')


plt.show()

result = poisson_blending(src,mask,target,100,50)

plt.figure(figsize=(60,60))

plt.subplot(3,1,1)
plt.imshow(src,cmap="gray")
plt.title('Source Image')
plt.axis('off')

plt.subplot(3,1,2)
plt.imshow(target,cmap="gray")
plt.title('Target Image')
plt.axis('off')

plt.subplot(3,1,3)
plt.imshow(result,cmap="gray")
plt.title('Result')
plt.axis('off')
plt.show()

In [ ]:
# Ejemplo 1 ---------> suponiendo vecinos constantes y no constantes
src = cv.imread("./ex1/src1.jpg",cv.IMREAD_GRAYSCALE)
mask = cv.imread("./ex1/mask.jpg",cv.IMREAD_GRAYSCALE)
target = cv.imread("./ex1/target.jpg",cv.IMREAD_GRAYSCALE)




result = poisson_blending(src,mask,target,0,50)
result2 = poisson_blending2(src,mask,target,0,50)

plt.figure(figsize=(60,60))

plt.subplot(1,2,1)
plt.imshow(result,cmap="gray")
plt.title('Source Image')
plt.axis('off')

plt.subplot(1,2,2)
plt.imshow(result2,cmap="gray")
plt.title('Target Image')
plt.axis('off')

plt.show()

In [ ]:
# Ejemplo 2 -----> falta normalizar la mascara
src = cv.imread("./ex1/src1.jpg",cv.IMREAD_GRAYSCALE)
mask = cv.imread("./ex1/mask.jpg",cv.IMREAD_GRAYSCALE)
target = cv.imread("./ex1/target2.jpg",cv.IMREAD_GRAYSCALE)

print(src.shape)
print(mask.shape)
print(target.shape)

cmap = ListedColormap(['black', 'red'])

plt.subplot(1,3,1)
plt.imshow(src,cmap="gray")
plt.title('Source Image')
plt.axis('off')

plt.subplot(1,3,2)
plt.imshow(mask,cmap=cmap)
plt.title('Mask')
plt.axis('off')

plt.subplot(1,3,3)
plt.imshow(target,cmap="gray")
plt.title('Target Image')
plt.axis('off')


plt.show()

result = poisson_blending(src,mask,target,820,370)

plt.figure(figsize=(60,60))

plt.subplot(3,1,1)
plt.imshow(src,cmap="gray")
plt.title('Source Image')
plt.axis('off')

plt.subplot(3,1,2)
plt.imshow(target,cmap="gray")
plt.title('Target Image')
plt.axis('off')

plt.subplot(3,1,3)
plt.imshow(result,cmap="gray")
plt.title('Result')
plt.axis('off')
plt.show()

In [ ]:
# Modificando la mascara para normalizarla, comparamos 2 metodos:
src = cv.imread("./ex1/src1.jpg",cv.IMREAD_GRAYSCALE)
mask = cv.imread("./ex1/mask.jpg",cv.IMREAD_GRAYSCALE)
target = cv.imread("./ex1/target.jpg",cv.IMREAD_GRAYSCALE)

print(np.unique(mask))

# mask = np.clip(mask,0,1)

mask = (mask > 180).astype(np.uint8)

print(np.unique(mask))

cmap = ListedColormap(['black', 'red'])

plt.subplot(1,3,1)
plt.imshow(src,cmap="gray")
plt.title('Source Image')
plt.axis('off')

plt.subplot(1,3,2)
plt.imshow(mask,cmap=cmap)
plt.title('Mask')
plt.axis('off')

plt.subplot(1,3,3)
plt.imshow(target,cmap="gray")
plt.title('Target Image')
plt.axis('off')


plt.show()

result_PB = poisson_blending(src,mask,target,100,50)
result_B = blend(src,mask,target,100,50)

plt.figure(figsize=(60,60))

plt.subplot(2,1,1)
plt.imshow(result_PB,cmap="gray")
plt.title('Poisson blending')
plt.axis('off')


plt.subplot(2,1,2)
plt.imshow(result_B,cmap="gray")
plt.title('Blend')
plt.axis('off')
plt.show()